In [1]:
# Import credentials
from resource_credentials import subscription_key, resource_group_name, workspace_name

## Create Workspace

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

# Create a ml client
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name)

# Define the workspace
ws = Workspace(name=workspace_name, location="eastus") 

ws = ml_client.workspaces.begin_create(ws).result()
print(ws)


### List workspaces

In [3]:
current_workspaces = ml_client.workspaces.list()
for workspace in current_workspaces:
    print(workspace.name)

dp100-certification
dp100-certification-sdk
